In [1]:
import json
from pathlib import Path
import os
import pandas as pd

In [2]:
!pip install s3fs

In [3]:
import s3fs

In [6]:
s3 = s3fs.S3FileSystem(
    anon=True,
    client_kwargs={
        'endpoint_url': 'https://github.com/abq215/dsc650'
      }
    )
df = pd.read_csv(
    s3.open('data/external/tidynomicon/site.csv', mode='rb')
 )
df2 = pd.read_csv(
    s3.open('data/external/tidynomicon/measurements.csv', mode='rb')
 )
df3 = pd.read_csv(
    s3.open('data/external/tidynomicon/person.csv', mode='rb')
 )
df4 = pd.read_csv(
    s3.open('data/external/tidynomicon/visited.csv', mode='rb')
 )

FileNotFoundError: Not Found

In [6]:
df=pd.read_csv('D:/VZW Twinsburg/Tuition assistance/Bellevue University_MSDS/DSC 650/DSC650_Big Data/dsc650/data/external/tidynomicon/site.csv')

In [7]:
df.head()

,site_id,latitude,longitude
0,DR-1,-49.85,-128.57
1,DR-3,-47.15,-126.72
2,MSK-4,-48.87,-123.40


In [10]:
def read_cluster_csv(file_path, endpoint_url='https://storage.budsc.midwest-datascience.com'):
    s3 = s3fs.S3FileSystem(
        anon=True,
        client_kwargs={
            'endpoint_url': endpoint_url
        }
    )
    return pd.read_csv(s3.open(file_path, mode='rb'))

current_dir = Path(os.getcwd()).absolute()
results_dir = current_dir.joinpath('results')
kv_data_dir = results_dir.joinpath('kvdb')
kv_data_dir.mkdir(parents=True, exist_ok=True)

people_json = kv_data_dir.joinpath('people.json')
visited_json = kv_data_dir.joinpath('visited.json')
sites_json = kv_data_dir.joinpath('sites.json')
measurements_json = kv_data_dir.joinpath('measurements.json')

In [11]:
class KVDB(object):
    def __init__(self, db_path):
        self._db_path = Path(db_path)
        self._db = {}
        self._load_db()

    def _load_db(self):
        if self._db_path.exists():
            with open(self._db_path) as f:
                self._db = json.load(f)

    def get_value(self, key):
        return self._db.get(key)

    def set_value(self, key, value):
        self._db[key] = value

    def save(self):
        with open(self._db_path, 'w') as f:
            json.dump(self._db, f, indent=2)

In [ ]:
def create_sites_kvdb():
    db = KVDB(sites_json)
    df = read_cluster_csv('data/external/tidynomicon/site.csv')
    for site_id, group_df in df.groupby('site_id'):
        db.set_value(site_id, group_df.to_dict(orient='records')[0])
    db.save()


def create_people_kvdb():
    db = KVDB(people_json)
    ## TODO: Implement code
    db.save()


def create_visits_kvdb():
    db = KVDB(visited_json)
    ## TODO: Implement code
    db.save()


def create_measurements_kvdb():
    db = KVDB(measurements_json)
    ## TODO: Implement code
    db.save()

In [ ]:
create_sites_kvdb()
create_people_kvdb()
create_visits_kvdb()
create_measurements_kvdb()